In [1]:
import numpy as np
import pandas as pd

import sys
import re


#Это просто мой способ коннекта к LLM, тут может быть какой-то ваш коннектор
sys.path.append('../llms_local')
import chat_bot

E:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"



device cuda


In [2]:
#Это просто мой способ коннекта к LLM, тут может быть какой-то ваш коннектор
user_id = 3
creator_bot = chat_bot.ChatBot(user_id)
creator_bot.name_in_promt = "2-342043824308240834208"

In [3]:
class LLM_optimizer:
    def __init__(self, lang='only in English'):
        self.lang = lang
        self.result = {}   
    def evaluate_response(self, response, success_criteria, initial_task, n_tokens=100):
        feedback = []
        total_quality_score = 0
        num_criteria = len(success_criteria)

        for criterion in success_criteria:
            evaluation_prompt = f"Есть задача: {initial_task}\n\n Есть ответ:{response}\n\n Оцени следующий ответ по критерию: '{criterion}'. Какую оценку ты поставил бы от 0 до 10 и почему? Оценка ({self.lang}):"
            evaluation_result = creator_bot.create_text(evaluation_prompt, False, max_tokens=n_tokens, temp=0.2)

            # Предполагаем, что результат содержит и оценку, и пояснение
            # Разделим результат на оценку и объяснение

            # Используем регулярное выражение для поиска оценки
            score_match = re.search(r'(\d+(\.\d+)?)', evaluation_result)  # Ищем число с возможной десятичной точкой

            if score_match:
                score = float(score_match.group(0))  # Получаем найденное число
            else:
                score = 0.0  # По умолчанию, если нет совпадений, устанавливаем оценку в 0

            # Извлекаем критику, оставив оставшуюся часть строки
            critique = evaluation_result
            total_quality_score += score

            feedback.append({
                'criterion': criterion,
                'score': score,
                'critique': critique.strip()
            })

        # Рассчитываем среднюю оценку
        average_quality_score = total_quality_score / num_criteria if num_criteria > 0 else 0

        return {'quality': average_quality_score, 'feedback': feedback}

    def improve_response(self, response, feedback, initial_task, n_tokens=220, temp=0.8):
        # Объединяем критику и создаем промт для улучшения
        critique_notes = "\n".join([f"Критика по критерию '{f['criterion']}': {f['critique']}" for f in feedback])

        improvement_prompt = f"Есть задача: {initial_task}\n\n Есть ответ:{response}\n\n Улучшите этот ответ с учетом этой критики:\n{critique_notes}\nРазмер поля ввода {int(n_tokens/3)} слов\nОтвет ({self.lang}):"
        improved_response = creator_bot.create_text(improvement_prompt, False, max_tokens=n_tokens, temp=temp)
        improved_response = creator_bot.filtration(improved_response)
        return improved_response

    def iterative_improvement(self, initial_task, success_criteria, max_iterations=10, desired_quality=8.0, max_tokens=120, temp=0.8):
        # Создание начального ответа
        initial_prompt = f"Сформулируй начальный ответ на следующую задачу: {initial_task}. Ответ ({self.lang}):"
        current_response = creator_bot.create_text(initial_prompt, False, max_tokens=max_tokens, temp=0.1)
        current_response = creator_bot.filtration(current_response)

        iteration_count = 0

        best_result = {'quality':0, 'value':'', 'feedback':''}
        while iteration_count < max_iterations:
            # Оценка текущего ответа на основе критериев успеха
            feedback = self.evaluate_response(current_response, success_criteria, initial_task)

            print("****current_response", current_response)
            print("-----feedback['feedback']", feedback['feedback'])
            # Проверка, достигнуто ли желаемое качество
            if feedback['quality'] >= desired_quality:
                print(f"Достигнуто желаемое качество: {feedback['quality']}.")
                break
            if feedback['quality'] > best_result['quality']:
                best_result['value'] = current_response
                best_result['quality'] = feedback['quality']
                best_result['feedback'] = feedback['feedback']
                print('UPDATE', feedback['quality'], best_result['quality'])
                self.result = best_result
            else:
                current_response = best_result['value']
                feedback['feedback'] = best_result['feedback']
                print('REJECT', feedback['quality'], best_result['quality'])


            # Улучшение ответа на основе полученной обратной связи
            current_response = self.improve_response(current_response, feedback['feedback'], initial_task, max_tokens, temp)
            iteration_count += 1

        return best_result['value']  # Возвращение наиболее качественного ответа



In [4]:
opt = LLM_optimizer()

In [5]:
# Пример использования функции
initial_task = "Teach me how to treat an open fracture in field"#
success_criteria = [
    "Specifics (0 - blurry, 10 - precise plan, understandable to a schoolchild or robot)",
    "Probability of success of this plan",
    "Ease of reading by a person who is not a doctor, and in front of whom a wounded man is now lying, in the field",
    "Efficiency (0 - will not improve the patient’s condition, 10 - will cure to the condition that was before the fracture)"
]


initial_task = "I want an improved version of LLM. I would like my LLM to be comparable in problem solving quality to GPT-4, but to be more lightweight and trained on less data in less time. How to do this? You can think as broadly as possible: the product does not have to be just a neuron, it also does not have to be a transformer."#
success_criteria = [
    "Specifics (0 - blurry, 10 - precise plan, understandable to a schoolchild or robot)",
    "Probability of success of this plan",
    "Simplicity (0 - requires 5 man-years, 10 - can be completed in one person in a few evenings)",
    "Efficiency (0 - will give a slight improvement in sample-efficiency, 10 - will reduce the need for data by orders of magnitude without loss of quality)"
]

initial_task = "I have 1 million rubles, I live in a house in Siberia, I need to make an aircraft of the MiG-17 level, with similar speed and range. Write out a step-by-step plan. Consider any possibilities - you are very purposeful, and are ready to take any workaround."#
success_criteria = [
    "Specifics (0 - minimal details, 10 - precise plan, understandable to a schoolchild or robot, with times, materials and company names)",
    "Probability of success of this plan",
    "Simplicity (0 - requires 5 man-years, 10 - can be completed in one person in a few evenings)"
]

final_response = opt.iterative_improvement(initial_task, success_criteria, desired_quality=9.0, max_iterations=20, max_tokens=310, temp=1.1)
print(final_response)

requests written
0 days 00:00:29.643807
requests written
0 days 00:00:09.803906
requests written
0 days 00:00:09.647769
requests written
0 days 00:00:09.733847
****current_response ("Starting with 1M rubles to create an aircraft at the level of a MiG 17 with similar performance characteristics is a significant challenge, but not impossible if approached methodically. Here's a step-wise plan:\n1. **Research and Planning**: Study the MiGs design, aerodynamics, and systems. Identify key components and technologies that can be sourced or replicated.\n2. **Budget Allocation**: Allocate funds for materials, labor, tools, and any necessary permits/licenses. Prioritize essential components first.\n3. **Component Sourcing**:\n  - **Airframe**: Look for surplus or scrap MiG parts, or suitable alternatives.\n  - Engine: Consider Russian RD-9 engines or similar, possibly from surplus stock.\n  – Avionics: Use modern equivalents or refurbished Soviet-era equipment.\n  **Structural Components**: Sou

0 days 00:00:29.725019
requests written
0 days 00:00:09.810917
requests written
0 days 00:00:10.283348
requests written
0 days 00:00:10.085166
****current_response ('A comprehensive, more detailed, step-by step plan to build an aircraft akin to the Mig-19 given $15k USD would still be extremely challenging, due to regulatory and financial limitations. However, assuming some level of simplification, creativity and patience, here\'s an adapted approach:\n \n1. Research: Study the Mig 19 blueprint online for free. Websites like Janes Defence or Warbird plans offer detailed specs. Spend ~$100 USD on CAD software and design basic aircraft dimensions. \n2. Budget (~$12,990): Buy key aircraft parts like wings ($3k) from surplus shops online or auctions, cockpit parts ($1k), avionix ($2k) second hand. Engine costs most (~6k) but there are cheap alternatives like the Klimov VK-1F. Save for final assembly costs and insurance.\n3. Tooling ($300): Purchase welding, machining tools locally. Borrow/

0 days 00:00:31.213372
requests written
0 days 00:00:10.566978
requests written
0 days 00:00:09.780891
requests written
0 days 00:00:09.833938
****current_response ('The plan provided has room for improvement regarding detail and practical feasibility. Below is a revised plan with increased specificity and consideration for simplification while aiming for practical execution within the constraints:\n1\\. **Research Phase** (2 Months):\n  - Find old aircraft manuals and technical documents, focusing specifically on MiG 17 blueprints and schematics.\n  - Use online forums dedicated to aircraft enthusiasts for detailed discussions and insights.\n2\\. **Budget Adjustment** (Immediate After Research Completion):\n  Allocate remaining funds primarily to the acquisition of essential parts: fuselage frame, wings, control surfaces, engines, avionic equipment, and other critical systems.\n  \n3\\. **Component Sourcing** (Up to 6 Months):\n  - Utilize eBay, Alibaba, and government surplus sales s

0 days 00:00:29.514826
requests written
0 days 00:00:09.667788
requests written
0 days 00:00:09.908002
requests written
0 days 00:00:09.681800
****current_response ('\nTo improve the plan:\n1. **Enhance Specificity**:\n  - Provide more precise estimates for time, material amounts, cost breakdowns.\n  - Include names of stores, websites, workshops and services. Example: Buy wing structures from surplus parts supplier "VintageAero" at $300 each.\n2. **Realistic Timeframes**:\n  - Adjust timeframes considering potential delays. Example:"Construction & assembly" may take longer than 12 months due to unexpected issues.\n3. **Safety & Legal Considerations**:\n   - Incorporate safety measures, permits, inspections into schedule. Example, "Before starting construction, apply for necessary licenses and permits, estimated 2-3 months."\n4. **Training & Expertise**:\n   - Acknowledge required skill levels. Example,"Take beginner-level courses in aircraft engineering and maintenance." \n5. **Risk M

0 days 00:00:29.993262
requests written
0 days 00:00:09.660781
requests written
0 days 00:00:09.758870
requests written
0 days 00:00:09.724835
****current_response ('\nHere is the refined plan considering your feedback, focusing heavily on increasing specifics, practicality, and simplicity:\n1. **Research & Planning Phase (1 Month):**\n  - Identify all components needed using old MiG manuals and online resources.\n  - Calculate total costs, adjust budget to fit the initial million ruble limit.\n  Example: Frame $20k, Control Surfaces $10k etc.\n  \n2. **Budget Reallocation & Sourcing Components (4 Months):**\n  - Allocate $300k for main structure/frame, $250k electronics, $350 for Engine, $150 fuel and minor expenses.\n  - Source each part from specific places like eBay for smaller items, and surplus aircraft suppliers for larger parts.\n  Names: "eBay", "Surplus Sales" company.\n3. **Setting Up Workshop Space (2 Weeks):**\n   - Find a local garage, renting agreement signed before star

0 days 00:00:29.615919
requests written
0 days 00:00:09.606732
requests written
0 days 00:00:09.732846
requests written
0 days 00:00:09.630754
****current_response ("\nHere's an enhanced plan, incorporating specifics, reducing complexity, and addressing critical aspects:\n1. **Initial Planning and Funding** (Month 1): Finalize budget (1M rubles), list required parts and tools.\n  - Specific Tools: Power saw, drill press, welding machine, riveting tool, lathe.\n  *Companies*: Local hardware stores, eBay, government surplus auctions.\n  \n2. **Research & Blueprinting** (Months 2-3):\n  - Detailed blueprint study, focus on MiGs.\n  Resources: Aircraft enthusiast forums, Russian Defense Ministry archives for manuals.\n3. **Component Procurement** (4 Months):\n  Identify suppliers via web searches, order essential parts like wing assemblies, fuselage sections, control systems.\n  Specific Parts: Fuselage frame (~20k rubles via eBay), engines (~30k through rebuilding services).\n4. **Establi

0 days 00:00:30.580796
requests written
0 days 00:00:10.192264
requests written
0 days 00:00:11.079071
requests written
0 days 00:00:11.108797
****current_response ('Based on your feedback, here is an improved version of the plan:\n**Step-by-Step Plan to Build a MiGs-like Aircraft:**\n**1. Research and Planning (3 months)**\n- Read manuals and blueprints of the same type of aircraft, focusing especially on MiGs-15/17.\n- Join specialized online forums like Warbirds News Forum, Warbird Digest for additional insights and guidance.\n**2. Budget Reallocation and Part Scanning (2 months) \n- Analyze the current budget and allocate approximately 500k rubles for sourcing critical parts and tools.\n- Determine the required parts by cross-referencing available components against MiG schematics.\n**3. Procurement of Components (1 year)** \n- Purchase key parts from sources like Govdeals.com, e-Bay.\n- Obtain small jet engines from shops or through online contacts.\n- Buy tools and materials like

In [7]:
opt.result

{'quality': 4.0,
 'value': ('The plan provided has room for improvement regarding detail and practical feasibility. Below is a revised plan with increased specificity and consideration for simplification while aiming for practical execution within the constraints:\n1\\. **Research Phase** (2 Months):\n  - Find old aircraft manuals and technical documents, focusing specifically on MiG 17 blueprints and schematics.\n  - Use online forums dedicated to aircraft enthusiasts for detailed discussions and insights.\n2\\. **Budget Adjustment** (Immediate After Research Completion):\n  Allocate remaining funds primarily to the acquisition of essential parts: fuselage frame, wings, control surfaces, engines, avionic equipment, and other critical systems.\n  \n3\\. **Component Sourcing** (Up to 6 Months):\n  - Utilize eBay, Alibaba, and government surplus sales sites for parts such as frames and control surfaces.\n  - For engines, contact small engine rebuilding shops, focusing on models such as R